# Séance 6 - Dimensionnement d'un correcteur PID pour réguler la vitesse de rotation d'une éolienne

## Objectifs du notebook
- Déterminer les paramètres d'un correcteur PID afin de répondre aux critères d'un cahier des charges pour réguler le comportement d'une éolienne.

In [2]:
from IPython.display import Image, display, Markdown # Package to work with Jupyter notebooks
from ipywidgets import interact, FloatSlider, HBox # Widget package for adding interactivity
from control import matlab as ml  # Python Control Systems Toolbox (compatibility with MATLAB)
import numpy as np              # Package to manipulate array and matrix
import matplotlib.pyplot as plt # Package to create figures and plots
import math # Package to be able to do some mathematical operations
import sys  # Package to be able to import local packages
import os   # Package to interact with the operating system
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd()))) # Add the current path to the system path to be able to import custom local packages
from regulabfct import ReguLabFct as rlf # Package useful for the regulation course of HELMo Gramme

## Régulation d'une éolienne

Nous voyons des champs d'éoliennes apparaître un peu partout dans le paysage. Néanmoins, ce moyen de production d'énergie verte dépend du vent qui est fortement irrégulier au niveau de sa puissance et de sa direction, alors qu'il est souhaitable de maintenir une vitesse de rotation du générateur électrique qu'elle contient constante afin de disposer d'une énergie électrique fiable et régulière. Afin de réussir à parvenir à cet objectif du mieux possible, on a recours à des systèmes de contrôle dont l'objectif est de réduire les effets de l'intermittence du vent et de ses changements de direction. Ceux-ci reposent sur l'ajustement de l'angle d'inclinaison des pales (appelé plus couramment *pitch*) afin de maintenir une vitesse de rotation constante.

Un modèle linéarisé de la vitesse de rotation d'un générateur en fonction du *pitch* des pales d'une éolienne est donné par :

$$H = \frac{4.2158(s-827.1)(s^2-5.489s+194.4)}{(s+0.195)(s^2+2.101s+482.6)}$$

Ce modèle correspond à celui d'une éolienne de 600 kW possédant une hauteur de 36,6 m, un diamètre de rotor de 40 m, une vitesse nominale de rotation du rotor de 41,7 tr/min, une vitesse nominale de rotation du générateur de 1800 tr/min et une vitesse d'inclinaison de pale de 18,7 °/s.

> Observe la stabilité du système en traçant la position de ses pôles et zéros à l'aide de la fonction `pzmap` ainsi que sa réponse fréquentielle avec le diagramme de Nichols.

> Écris ton code pour réaliser cela dans la cellule suivante.


In [3]:
# [À compléter par toi !]
H_eol1 = ml.tf([4.2158],[1,0.195])
H_eol2 = ml.tf([1,-827.1],[1,2.101,482.6])
H_eol3 = ml.tf([1,-5.489,194.4],[1])
H_eol = H_eol1 * H_eol2 * H_eol3
print(H_eol)
rlf.pzmap(H_eol)


4.216 s^3 - 3510 s^2 + 1.996e+04 s - 6.779e+05
----------------------------------------------
       s^3 + 2.296 s^2 + 483 s + 94.11



(Figure({
     'data': [{'legendgroup': 'sys1',
               'legendgrouptitle': {'text': 'sys1'},
               'marker': {'color': '#1F77B4', 'line': {'color': '#1F77B4', 'width': 2}, 'size': 10, 'symbol': 'x-thin'},
               'mode': 'markers',
               'name': 'Poles',
               'type': 'scatter',
               'x': array([-1.0505, -1.0505, -0.195 ]),
               'y': array([ 21.94302736, -21.94302736,   0.        ])},
              {'legendgroup': 'sys1',
               'marker': {'color': 'rgba(255, 255, 255, 0)',
                          'line': {'color': '#1F77B4', 'width': 2},
                          'size': 10,
                          'symbol': 'circle'},
               'mode': 'markers',
               'name': 'Zeros',
               'type': 'scatter',
               'x': array([827.1   ,   2.7445,   2.7445]),
               'y': array([  0.        ,  13.66995683, -13.66995683])}],
     'layout': {'legend': {'groupclick': 'toggleitem'},
          

## Simplification du modèle

Comme on peut le voir dans le plan complexe de Laplace, bien que le modèle soit instable suite à ses zéros à partie réelle positives, ceux-ci sont fortement dominés par un pôle présentant une constante de temps nettement plus grande (beaucoup plus que 3-4 fois plus grande). Dès lors, leur impact n'a d'effet que très peu de temps au début du régime transitoire et peut probablement être ignoré. Nous pouvons donc simplifier notre modèle par un modèle du premier ordre qui soit stable en se limitant à son pôle dominant. Nous obtenons ainsi la fonction de transfert simplifiée suivante :

$$\frac{K}{\tau \cdot s + 1} \quad \text{avec} \quad \begin{cases} 
K = \frac{4.2158 \cdot (-827.1) \cdot 194.4}{0.195 \cdot 482.6} \approx -7200\\ 
\tau = \frac{1}{0.195} \approx 5
\end{cases}$$

***ATTENTION :*** Bien que nous simplifions le système à l'impact de son pôle dominant, nous devons prendre en compte les impacts des différents gains statiques du modèle complet !

> Dimensionne le correcteur qu'il faudrait pour assurer une **erreur de position nulle** tout en assurant un temps de stabilisation $(t_{s_{5\%}})$ inférieur à **1 seconde** et un **dépassement relatif inférieur à 25%**.

> Afin de vérifier le bon fonctionnement du correcteur que tu as dimensionné, compare le tracé de la réponse indicielle du système simplifié avec celui du modèle complet.


In [7]:
# [À compléter par toi !]
P_simpl = ml.tf([-7200],[5,1])
P_reel = H_eol
Kp = -0.0021
PI_s = Kp * ml.tf([5,1],[5,0])
GBO_P_simpl = PI_s * P_simpl
GBO_P_reel = PI_s * P_reel
GBF_P_simpl = ml.feedback(GBO_P_simpl,1)
GBF_P_reel = ml.feedback(GBO_P_reel,1)
rlf.step([GBF_P_simpl, GBF_P_reel ], plot_infos=True);